In [1]:
from tkinter import * 
from tkinter import messagebox
from tkinter import filedialog
import cv2
from PIL import Image, ImageTk

import string
import re
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer
import pandas as pd

from sklearn.naive_bayes import MultinomialNB
from imblearn.over_sampling import RandomOverSampler


from sklearn.naive_bayes import MultinomialNB
import os

In [2]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\AMI\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
win=Tk()
win.state("zoomed")
win.resizable(width=False,height=False)
win.configure(bg="orange")
win.title("My project")
lbl_title=Label(win,text="News Classification",font=('',55,'bold','underline'),bg='orange')
lbl_title.place(relx=.28,rely=0)

In [4]:
spwords=stopwords.words("english")
spwords.remove('not')
spwords.remove("don't")
updated_spwords=spwords[:-34]
wnl=WordNetLemmatizer()

def text_cleaning(doc):
    doc=doc.lower()
    doc=re.sub(f"[{string.punctuation}]","",doc)
    ss=""
    for w in word_tokenize(doc):
        if(w not in updated_spwords):
            ss=ss+" "+wnl.lemmatize(w)
    return ss.strip()





df=pd.read_json(r"D:\Adity_sir\data\sentiment\news.json",lines=True)
corpus=df.headline
y=df.category
final_corpus=corpus.apply(text_cleaning)
cv=CountVectorizer(binary=False,ngram_range=(1,3))
X=cv.fit_transform(final_corpus)
ros1 = RandomOverSampler(random_state=42)

X_res1, y_res1 = ros1.fit_resample(X, y)

model=MultinomialNB()
model.fit(X_res1,y_res1)



MultinomialNB()

In [5]:
def one_news_pred_screen():
    
    def news_prediction():
        test=entry_for_news.get()
        sample=str(test)
        if(len(sample)==0):  #------>empty check
                messagebox.showerror("Empty Value","check added data")
        else:

                pred_sample=text_cleaning(sample)
                pred=cv.transform([pred_sample]).toarray()
                ans=model.predict(pred)[0]
                prediction=str(ans)
                
                
               
                
                
                lbl_pred=Label(frm,text=prediction,font=('',15,'bold'),bg='sky blue')
                lbl_pred.place(relx=.50,rely=.46)
                lbl_pred_text=Label(frm,text="Prediction:",font=('',18,'bold'),bg='sky blue')
                lbl_pred_text.place(relx=.38,rely=.45)
                
                
                
                

    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
 
    lbl_one_news=Label(frm,text="Enter news",font=('',15,'bold'),bg='sky blue')
    lbl_one_news.place(relx=.3,rely=.23)

    entry_for_news=Entry(frm,font=('',20,'bold'),bd=10)
    entry_for_news.place(relx=.42,rely=.22)
    
    btn_pred=Button(frm,command=lambda:news_prediction(),text="Predict",font=('',20,'bold'),bd=10,width=8)
    btn_pred.place(relx=.40,rely=.32)


    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    
                
    
    
    

In [6]:
iFrame=None
def browse_file():
        global iFrame
        if(iFrame!=None):
            iFrame.destroy()
        file_path=filedialog.askopenfilename()
        file = open(file_path, "r")
        lines=file.readlines()

        for i in range(0,len(lines)):
            sample=lines[i]
            pred_sample=text_cleaning(sample)
            pred=cv.transform([pred_sample]).toarray()
            #if (os.path.exists ("prediction")):
                   #pass
            #else:
                #os.mkdir("prediction")
            path=file_path
            new_path = path.rsplit('/', 1)[0]
            path_pred=new_path+"/"+"pred.txt"


            file=open(path_pred,"a")
            j=model.predict(pred)[0]
            file.write(lines[i]+"-"+j+"\n")
            file.close()
            
        messagebox.showinfo("validation","File Created in same Directory")
        file.close()
        

In [7]:


def file_news_pred():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
  
    btn_Browse=Button(frm,command=lambda:browse_file(),text="Browse",font=('',20,'bold'),bd=10,width=8)
    btn_Browse.place(relx=.37,rely=.05)
    
    
    btn_back=Button(frm,command=lambda:welcome_screen(),text="back",font=('',20,'bold'),bd=10)
    btn_back.place(relx=.9,rely=0)
    
    

In [8]:
def home_screen():
    frm=Frame(win,bg='sky blue')
    frm.place(relx=0,rely=.12,relwidth=1,relheight=1)
    
    lbl_user=Label(frm,text="Username",font=('',20,'bold'),bg='sky blue')
    lbl_user.place(relx=.3,rely=.3)

    entry_user=Entry(frm,font=('',20,'bold'),bd=10)
    entry_user.place(relx=.42,rely=.3)
    entry_user.focus()

    lbl_pass=Label(frm,text="Password",font=('',20,'bold'),bg='sky blue')
    lbl_pass.place(relx=.3,rely=.4)

    entry_pass=Entry(frm,font=('',20,'bold'),bd=10,show="*")
    entry_pass.place(relx=.42,rely=.4)

    btn_login=Button(frm,command=lambda:welcome_screen(entry_user,entry_pass),text="login",font=('',20,'bold'),bd=10,width=10)
    btn_login.place(relx=.45,rely=.5)

def welcome_screen(entry_user=None,entry_pass=None):
    if(entry_user!=None and entry_pass!=None):
        user=entry_user.get()
        pwd=entry_pass.get()
    else:
        user="admin"
        pwd="admin"
    if(len(user)==0 or len(pwd)==0):
        messagebox.showwarning("validation","Please fill both fields")
        return
    else:
        if(user=="admin" or pwd=="admin"):
            frm=Frame(win,bg='skyblue')
            frm.place(relx=0,rely=.15,relwidth=1,relheight=1)
            
            
            btn_image_only_news=Button(frm,command=lambda:one_news_pred_screen(),text="News Prediction",font=('',20,'bold'),bd=10,width=20)
            btn_image_only_news.place(relx=.40,rely=.2)
            
            btn_image_file=Button(frm,command=lambda:file_news_pred(),text="News File Prediction",font=('',20,'bold'),bd=10,width=20)
            btn_image_file.place(relx=.40,rely=.4)
            
            
        else:
            messagebox.showerror("Fail","Invalid Username/Password")    
    
    
home_screen()
win.mainloop()
